In [ ]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
import os
def get_tiff_metadata(tiff_path):
    with rasterio.open(tiff_path) as src:
        width = src.width
        height = src.height
        transform = src.transform
        crs = src.crs
        return width, height, transform, crs

def create_tiff_from_shapefile(input_shapefile, output_tiff, reference_tiff):
    # Load the shapefile
    shapes = gpd.read_file(input_shapefile)

    # Get metadata from the reference TIFF
    width, height, transform, crs = get_tiff_metadata(reference_tiff)

    # Rasterize the shapefile to match the dimensions of the reference TIFF
    rasterized_output = rasterize(
        [geom for geom in shapes.geometry],
        out_shape=(height, width),
        fill=0,
        transform=transform,
        dtype='int32'
    )

    # Save the rasterized output to a TIFF file
    with rasterio.open(
        output_tiff, 'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=str(rasterized_output.dtype),
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(rasterized_output, 1)

    print("TIFF file created successfully.")

# Specify the path to your TIFF file
tiff_path = 'ThanhHoa_SubImage0_raster.tif'

# Create tiff mask
create_tiff_from_shapefile(
    'Image0_wheat.shp',
    'output.tif',
    tiff_path  # Reference the original TIFF file to match its dimensions
)


In [ ]:
for tiff_file in os.listdir('/'):
    if tiff_file.endswith('.shp'):
        shapefile_path = os.path.join('data', tiff_file)
        output_tiff_path = os.path.join('/', tiff_file.split('.')[0] + '_label.tif')
        reference_tiff_path = os.path.join('data', "Sub"+ tiff_file.split('_')[0] + '.tif')

        create_tiff_from_shapefile(shapefile_path, output_tiff_path, reference_tiff_path)
        print(f"Created mask for {tiff_file}.")